In [159]:
pip install torch torchvision

In [160]:
import torch
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms

In [161]:
transform = transforms.Compose([
transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [162]:
dataset = ImageFolder(root='data', transform=transform)

In [163]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
#untested = len(dataset) - test_size- train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [164]:
#trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [165]:
nc = len(dataset.classes)

In [166]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split

In [167]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        #self.pool = nn.MaxPool2d(2,2)
        #self.conv3 = nn.Conv2d(16, 30, 5)
        self.fc1 = nn.Linear(16 * 5 * 5 , 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2) 

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
       # x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [168]:
model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr=0.01)

In [169]:
Accuracies = []

In [170]:
for epoch in range(10):
    model.train()
    running_loss = 0.0
    total_correct = 0
    total_samples = 0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate accuracy
        total_correct += (outputs.argmax(1) == labels).sum().item()
        total_samples += labels.size(0)

        acc = total_correct/total_samples

        if i % 10 == 0:
            Accuracies.append(acc)
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 10:.3f}, Accuracy: {acc:.3f}')
            running_loss = 0.0
            
            
print('Finished Training')

[Epoch 1, Batch 1] Loss: 0.071, Accuracy: 0.250
[Epoch 2, Batch 1] Loss: 0.065, Accuracy: 0.875
[Epoch 3, Batch 1] Loss: 0.052, Accuracy: 0.906
[Epoch 4, Batch 1] Loss: 0.058, Accuracy: 0.781
[Epoch 5, Batch 1] Loss: 0.034, Accuracy: 0.906
[Epoch 6, Batch 1] Loss: 0.037, Accuracy: 0.875
[Epoch 7, Batch 1] Loss: 0.048, Accuracy: 0.812
[Epoch 8, Batch 1] Loss: 0.043, Accuracy: 0.844
[Epoch 9, Batch 1] Loss: 0.042, Accuracy: 0.781
[Epoch 10, Batch 1] Loss: 0.019, Accuracy: 0.969
Finished Training


In [171]:
def avg_acc():
    avgAcc = 0
    sumAcc = 0
    for num in Accuracies:
        sumAcc += num
    avgAcc = sumAcc/len(Accuracies)
    print(f'Average Training Accuracy: {acc*100:.2f}')

In [172]:
avg_acc()

Average Training Accuracy: 87.18


In [173]:
model.eval()

SimpleCNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=2, bias=True)
)

In [174]:
def evaluate_model(model, dataloader):
    avg_acc()
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    print(f'Testing Accuracy: {accuracy*100:.2f}')

In [175]:
evaluate_model(model, testloader)
print('Finished Training and Evaluation')

Average Training Accuracy: 87.18
Testing Accuracy: 90.00
Finished Training and Evaluation


In [176]:
torch.save(model.state_dict(), 'cat_dog_cnn.pth')

In [177]:
pip install onnx 

Note: you may need to restart the kernel to use updated packages.


In [178]:
exinput = torch.randn(1, 3, 32, 32)
torch.onnx.export(model, exinput, 'cat_dog_cnn.onnx', verbose=True, input_names=['input'], output_names=['output'])
print('Model trained and exported to ONNX format.')

Model trained and exported to ONNX format.
